In [4]:
import torchkbnufft as tkbn


In [5]:
def inufft2c_torch(data, ktraj, sens, dim=(-2, -1)):
    # JBM implemented
    inufft_ob = tkbn.KbNufftAdjoint(im_size=dim)
    image = inufft_ob(data, ktraj, smaps=sens, norm='ortho')
    print('inufft Not tested')
    return image
    #return torch.fft.fftshift(torch.fft.ifft2(torch.fft.ifftshift(data, dim), dim=dim), dim=dim)

def nufft2c_torch(data, ktraj, sens, dim=(-2, -1)):
    # JBM implemented
    nufft_ob = tkbn.KbNufft(im_size=dim)
    kdata = nufft_ob(data, ktraj, smaps=sens, norm='ortho')
    print('nufft Not tested')
    return kdata
    #return torch.fft.fftshift(torch.fft.ifft2(torch.fft.ifftshift(data, dim), dim=dim), dim=dim)